__Download MNIST data__

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


__Example image from test data__

In [2]:
from PIL import Image
from io import BytesIO
import IPython.display
import numpy as np


def display(img):
    img = img.reshape((28,28)) * 255

    f = BytesIO()
    img = np.uint8(img)
    Image.fromarray(img).save(f, 'png')

    IPython.display.display(IPython.display.Image(data=f.getvalue()))
    
img = mnist.test.images[0]
display(img)

In [3]:
import tensorflow as tf
BATCH_SIZE = 10
LABEL_SIZE = 10

_, IMAGE_PIXELS = mnist.train.images.shape
# placeholder for input data
x = tf.placeholder(tf.float32, shape=(None, IMAGE_PIXELS))
# placeholder for input correct answers
t = tf.placeholder(tf.float32, shape=(None, LABEL_SIZE))

__Create hidden layers & output layer__

In [4]:
import math
HIDDEN1_UNITS = 50
HIDDEN2_UNITS = 30

with tf.name_scope('hidden1'):
    W1 = tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS, HIDDEN1_UNITS],
                                 stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='h1_weights')
    b1 = tf.Variable(tf.zeros([HIDDEN1_UNITS]),
                     name='h1_bias')
    
with tf.name_scope('hidden2'):
    W2 = tf.Variable(
            tf.truncated_normal([HIDDEN1_UNITS, HIDDEN2_UNITS]),
            name='h2_weights')
    b2 = tf.Variable(tf.zeros([HIDDEN2_UNITS]),
                     name='h2_bias')
    
with tf.name_scope('output'):
    W3 = tf.Variable(
            tf.truncated_normal([HIDDEN2_UNITS, LABEL_SIZE]),
            name='o_weights')
    b3 = tf.Variable(tf.zeros([LABEL_SIZE]),
                     name='_obias')

h1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
h2 = tf.nn.sigmoid(tf.matmul(h1, W2) + b2)
y = tf.nn.softmax(tf.matmul(h2, W3) + b3)

__Loss__

In [5]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

__Setup session and run__

In [6]:
def fill_feed_dict(data_set):
    images_feed, labels_feed = data_set.next_batch(BATCH_SIZE)
    feed_dict = {
      x: images_feed,
      t: labels_feed,
    }
    return feed_dict

In [7]:
def estimate(data_set):
    feed_dict = fill_feed_dict(data_set)
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(t,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [ ]:
MAX_STEPS = 1000

session = tf.InteractiveSession()
init = tf.global_variables_initializer()
session.run(init)

for step in range(1, MAX_STEPS + 1):
    accuracy = estimate(mnist.validation)
    feed_dict = fill_feed_dict(mnist.train)
    _, acc_rate = session.run([train_step, accuracy], feed_dict=feed_dict)
    if step == 1 or step == MAX_STEP or step % 100 == 0:
        print('Step %d: accuracy = %.2f' % (step, acc_rate))

Step 1: accuracy = 0.10
Step 100: accuracy = 0.80
Step 200: accuracy = 0.90
Step 300: accuracy = 0.80
Step 400: accuracy = 0.90
Step 500: accuracy = 0.90
Step 600: accuracy = 0.80
Step 700: accuracy = 0.80
Step 800: accuracy = 1.00
Step 900: accuracy = 0.70

__Evaluate__

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
result = session.run(accuracy, feed_dict={x: mnist.test.images, t: mnist.test.labels})   
print("Accuracy is: %.2f%%" %(result * 100))

In [ ]:
# close session to free resources
session.close()